# AI-Mind

[Minds](https://mindsdb.com/minds) are AI systems provided by [MindsDB](https://mindsdb.com/) that work similarly to large language models (LLMs) but go beyond by answering any question from any data. 

This is accomplished by selecting the most relevant data for an answer using parametric search, understanding the meaning and providing responses within the correct context through semantic search, and finally, delivering precise answers by analyzing data and using machine learning (ML) models.

The `AIMindTool` can be used to query data sources in natural language via LangChain by simply configuring their connection parameters.

## Overview

### Integration details

- TODO: Make sure links and features are correct


### Tool features

- TODO: Add feature table if it makes sense


## Setup

To access Minds via the `AIMindTool`, you'll need to create a Minds account, get an API key, and install the `langchain-minds` integration package.

In [ ]:
%pip install --quiet -U langchain-minds

### Credentials

In [1]:
import getpass
import os

if not os.environ.get("MINDS_API_KEY"):
    os.environ["MINDS_API_KEY"] = getpass.getpass("MINDS API key:\n")

if not os.environ.get("POSTGRES_PASSWORD"):
    os.environ["POSTGRES_PASSWORD"] = getpass.getpass("Postgres password:\n")

## Instantiation

In [2]:
from langchain_minds import AIMindDataSource, AIMindEnvVar, AIMindAPIWrapper, AIMindTool


# Create a data source that your Mind will have access to.
# This will create a new data source using the connection data provided with the given name.
# To configure additional data sources, simply create additional instances of
# AIMindDataSource and pass it to the wrapper below.
datasource = AIMindDataSource(
    name="demo_datasource",
    description="house sales data",
    engine="postgres",
    connection_data={
        "user": 'demo_user',
        "password": AIMindEnvVar('POSTGRES_PASSWORD', is_secret=True), # Use an environment variable for the password.
        "host": "samples.mindsdb.com",
        "port": 5432,
        "database": "demo",
        "schema": "demo_data"
    },
    tables=['house_sales']
)

# NOTE: Feel free to use the above connection data as is! It's our demo database open to the public.
# The password is "demo_password".

# To re-use an existing data source, simply provide the name of the data source without any other parameters.
# datasource = AIMindDataSource(
#     name="demo_datasource",
# )

# Create the wrapper for the Minds API by giving the Mind a name and passing in the data sources created above.
# This will create a new Mind with the given name and access to the data sources.
api_wrapper = AIMindAPIWrapper(
    name="demo_mind",
    datasources=[datasource]
)

# To re-use an existing Mind, simply provide the name of the Mind without any data sources.
# aimind_tool = AIMindAPIWrapper(
#     name="demo_mind",
# )

# Create the tool by simply passing in the API wrapper from before.
tool = AIMindTool(
    api_wrapper=api_wrapper
)

/home/minura/anaconda3/envs/langchain_minds/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## Invocation

### Invoke directly with args

The only required argument to query the `AIMindTool` is `query`, which is a question for the Mind in plain English.

In [3]:
tool.invoke({"query": "How many three-bedroom houses were sold in 2008?"})

'The number of three-bedroom houses sold in 2008 was 8.'

### Invoke with ToolCall

We can also invoke the tool with a model-generated ToolCall, in which case a ToolMessage will be returned.

In [4]:
# This is usually generated by a model, but we'll create a tool call directly for demo purposes.
model_generated_tool_call = {
    "args": {"query": "How many three-bedroom houses were sold in 2008?"},
    "id": "1",
    "name": tool.name,
    "type": "tool_call",
}
tool.invoke(model_generated_tool_call)

ToolMessage(content='The query has been executed successfully. A total of 8 three-bedroom houses were sold in 2008.', name='ai_mind', tool_call_id='1')

## Chaining

- TODO: Add example of using the tool with a tool calling model in a chain


## API reference

For detailed documentation of all Minds features and configurations head to the API reference: 
- TODO: add API reference link